In [182]:
import numpy as np
import torch
from torch import nn
import torchvision
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchsummary import summary

In [194]:
# Load the Fashion MNIST dataset using torchvision
transform = transforms.Compose([
    transforms.ToTensor()
])
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True,transform=transform, download=False)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=transform, download=False)

In [180]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder,self).__init__()
        self.encoder = nn.Sequential(
                nn.Conv2d(1,32,3, stride=2, padding=1),
                          nn.ReLU(),
                          nn.Conv2d(32,64, 3, stride=2, padding=1),
                          nn.ReLU(),
                          nn.Conv2d(64,128,3,stride=2, padding=1),
                          nn.ReLU(),
                        nn.Flatten(start_dim=1),
                         nn.Linear(2048,2),
                         nn.ReLU(),)
        self.decoder=nn.Sequential(
            nn.Linear(2,2048),
            nn.Unflatten(1,(128,4,4)),
            nn.ConvTranspose2d(128, 128, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64,32,4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32,1,3, padding='same'),
            nn.Sigmoid()
            
        )

    def forward(self,x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
        
            
            
            
         
model=Autoencoder()
summary(model, (1,32,32))   

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 16, 16]             320
              ReLU-2           [-1, 32, 16, 16]               0
            Conv2d-3             [-1, 64, 8, 8]          18,496
              ReLU-4             [-1, 64, 8, 8]               0
            Conv2d-5            [-1, 128, 4, 4]          73,856
              ReLU-6            [-1, 128, 4, 4]               0
           Flatten-7                 [-1, 2048]               0
            Linear-8                    [-1, 2]           4,098
              ReLU-9                    [-1, 2]               0
           Linear-10                 [-1, 2048]           6,144
        Unflatten-11            [-1, 128, 4, 4]               0
  ConvTranspose2d-12            [-1, 128, 8, 8]         262,272
             ReLU-13            [-1, 128, 8, 8]               0
  ConvTranspose2d-14           [-1, 64,

# Defining the Training Loop

In [198]:
def train(model,num_epochs=5, batch_size=64, learning_rate=1e-3):
    torch.manul_seed(42)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(),
                            lr = learning_rate,
                            weight_decay = 1e-5,)
    train_loader= torch.utils.data.DataLoader(train_dataset)
    outputs = []
    for epoch in range(num_epochs):
        for data in train_loader:
            img, _ = data
            recon  = model(img)
            loss  = criterion(recon, img)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch:{}, Loss:{:.4f}'.format(eoch+1, float(loss)))
        outputss.append((epoch, img, recon))
    return outputs